In [1]:
# !~/tf-nvidia/bin/pip3 install git+https://github.com/huseinzol05/tensor2tensor.git --no-deps

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry

2022-07-25 20:41:22.867767: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0



/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from malaya.text.t2t import text_encoder
import malaya

In [5]:
encoder = text_encoder.SubwordTextEncoder('en-ms.subwords')

In [6]:
class Encoder:
    def __init__(self, encoder):
        self.encoder = encoder
        self.vocab_size = encoder.vocab_size

    def encode(self, s):
        s = [self.encoder.encode(s_) for s_ in s]
        s = [i + [1] for i in s]
        return s

    def decode(self, ids, strip_extraneous = False):
        return self.encoder.decode(ids)

In [7]:
s_encoder = Encoder(encoder)

In [8]:
@registry.register_problem
class Translation(text_problems.Text2TextProblem):
    @property
    def approx_vocab_size(self):
        return encoder.vocab_size

    @property
    def is_generate_per_split(self):
        return False

    @property
    def dataset_splits(self):
        return [
            {'split': problem.DatasetSplit.TRAIN, 'shards': 100},
            {'split': problem.DatasetSplit.EVAL, 'shards': 1},
        ]

    def feature_encoders(self, data_dir):
        s_encoder = Encoder(encoder)
        return {'inputs': s_encoder, 'targets': s_encoder}

In [9]:
PROBLEM = 'translation'
problem = problems.problem(PROBLEM)

In [10]:
class Model:
    def __init__(self, HPARAMS = "transformer_base", DATA_DIR = 't2t/data2'):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.beam_size = tf.placeholder(tf.int32, shape=(), name = 'beam_size')
        self.temperature = tf.placeholder(tf.float32, shape=(), name = 'temperature')
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        maxlen_decode = 50 + tf.reduce_max(self.X_seq_len)
        
        x = tf.expand_dims(tf.expand_dims(self.X, -1), -1)
        y = tf.expand_dims(tf.expand_dims(self.Y, -1), -1)
        
        features = {
            "inputs": x,
            "targets": y,
            "target_space_id": tf.constant(1, dtype=tf.int32),
        }
        print(features)
        
        Modes = tf.estimator.ModeKeys
        hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)
        translate_model = registry.model('transformer')(hparams, Modes.PREDICT)
        logits, _ = translate_model(features)
        
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
            self.fast_result = translate_model._greedy_infer(features, maxlen_decode)["outputs"]
            self.beam_result = translate_model._beam_decode_slow(
                features, maxlen_decode, beam_size=self.beam_size, 
                top_beams=1, alpha=self.temperature)["outputs"]
        
        self.fast_result = tf.identity(self.fast_result, name = 'greedy')
        self.beam_result = tf.identity(self.beam_result, name = 'beam')

In [11]:
import tensorflow as tf

In [12]:
ckpt_path = tf.train.latest_checkpoint('t2t-noisy-en-ms/base')
ckpt_path

't2t-noisy-en-ms/base/model.ckpt-775000'

In [13]:
!ls t2t-noisy-en-ms/base/

checkpoint
eval
events.out.tfevents.1658537515.huseincomel-desktop
events.out.tfevents.1658566826.huseincomel-desktop
events.out.tfevents.1658570269.huseincomel-desktop
graph.pbtxt
model.ckpt-300000.data-00000-of-00002
model.ckpt-300000.data-00001-of-00002
model.ckpt-300000.index
model.ckpt-300000.meta
model.ckpt-325000.data-00000-of-00002
model.ckpt-325000.data-00001-of-00002
model.ckpt-325000.index
model.ckpt-325000.meta
model.ckpt-350000.data-00000-of-00002
model.ckpt-350000.data-00001-of-00002
model.ckpt-350000.index
model.ckpt-350000.meta
model.ckpt-375000.data-00000-of-00002
model.ckpt-375000.data-00001-of-00002
model.ckpt-375000.index
model.ckpt-375000.meta
model.ckpt-400000.data-00000-of-00002
model.ckpt-400000.data-00001-of-00002
model.ckpt-400000.index
model.ckpt-400000.meta
model.ckpt-425000.data-00000-of-00002
model.ckpt-425000.data-00001-of-00002
model.ckpt-425000.index
model.ckpt-425000.meta
model.ckpt-450000.data-00000-of-00002
model.ckpt-450000.data-00001-of-00002
model

In [14]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

2022-07-25 20:41:28.979939: I tensorflow/core/platform/profile_utils/cpu_utils.cc:109] CPU Frequency: 2496000000 Hz
2022-07-25 20:41:28.980322: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x8f6a1d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-07-25 20:41:28.980339: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-07-25 20:41:28.981831: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-25 20:41:28.985518: E tensorflow/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-25 20:41:28.985535: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-07-25 20:41:28.985539: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel

{'inputs': <tf.Tensor 'ExpandDims_1:0' shape=(?, ?, 1, 1) dtype=int32>, 'targets': <tf.Tensor 'ExpandDims_3:0' shape=(?, ?, 1, 1) dtype=int32>, 'target_space_id': <tf.Tensor 'Const_1:0' shape=() dtype=int32>}
INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting hparams.dropout to 0.0


INFO:tensorflow:Setting hparams.dropout to 0.0


...

INFO:tensorflow:Transforming feature 'targets' with symbol_modality_26088_512.targets_bottom


INFO:tensorflow:Building model body


INFO:tensorflow:Building model body


INFO:tensorflow:Transforming body output with symbol_modality_26088_512.top


INFO:tensorflow:Transforming body output with symbol_modality_26088_512.top


In [15]:
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, 't2t-noisy-en-ms/base/model.ckpt-775000')

INFO:tensorflow:Restoring parameters from t2t-noisy-en-ms/base/model.ckpt-775000


INFO:tensorflow:Restoring parameters from t2t-noisy-en-ms/base/model.ckpt-775000


In [16]:
string = '(CNN)New York Attorney General Letitia James on Monday ordered the Black Lives Matter Foundation -- which she said is not affiliated with the larger Black Lives Matter movement -- to stop collecting donations in New York. "I ordered the Black Lives Matter Foundation to stop illegally accepting donations that were intended for the #BlackLivesMatter movement. This foundation is not affiliated with the movement, yet it accepted countless donations and deceived goodwill," James tweeted.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded], model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

2022-07-25 20:41:51.755785: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 53428224 exceeds 10% of system memory.
2022-07-25 20:41:51.766420: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 53428224 exceeds 10% of system memory.
2022-07-25 20:41:51.766420: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 53428224 exceeds 10% of system memory.
2022-07-25 20:41:51.767743: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 53428224 exceeds 10% of system memory.
2022-07-25 20:41:51.837478: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 53428224 exceeds 10% of system memory.


('(CNN) Peguam Negara New York Letitia James pada hari Isnin memerintahkan Yayasan Black Lives Matter - yang menurutnya tidak berafiliasi dengan gerakan Black Lives Matter yang lebih besar - untuk berhenti mengumpulkan sumbangan di New York. "Saya memerintahkan Yayasan Black Lives Matter untuk berhenti menerima sumbangan secara haram yang ditujukan untuk gerakan #BlackLivesMatter. Yayasan ini tidak berafiliasi dengan gerakan itu, namun ia menerima sumbangan yang tak terhitung jumlahnya dan muhibah yang ditipu," tweet James.<EOS>',
 '(CNN) Peguam Negara New York Letitia James pada hari Isnin memerintahkan Yayasan Black Lives Matter - yang menurutnya tidak berafiliasi dengan gerakan Black Lives Matter yang lebih besar - untuk berhenti mengumpulkan sumbangan di New York. "Saya memerintahkan Yayasan Black Lives Matter untuk berhenti menerima sumbangan secara haram yang ditujukan untuk gerakan #BlackLivesMatter. Yayasan ini tidak berafiliasi dengan gerakan itu, namun ia menerima banyak sumb

In [17]:
string = 'WHEN ALICIA GARZA wrote "Black lives matter" in a Facebook post nearly seven years ago, the activist from Oakland, California, never imagined that those words would come to define a global movement. The July 13, 2013, acquittal of George Zimmerman for the killing of Trayvon Martin, an unarmed Black teenager, sparked her post.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded],model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

('KETIKA ALICIA GARZA menulis "Perkara hidup hitam" dalam catatan Facebook hampir tujuh tahun yang lalu, aktivis dari Oakland, California, tidak pernah membayangkan bahawa kata-kata itu akan menentukan pergerakan global. 13 Julai 2013, pembebasan George Zimmerman atas pembunuhan Trayvon Martin, seorang remaja Hitam yang tidak bersenjata, mencetuskan jawatannya.<EOS>',
 'KETIKA ALICIA GARZA menulis "Perkara hidup hitam" dalam catatan Facebook hampir tujuh tahun yang lalu, aktivis dari Oakland, California, tidak pernah membayangkan bahawa kata-kata itu akan menentukan pergerakan global. 13 Julai 2013, pembebasan George Zimmerman atas pembunuhan Trayvon Martin, seorang remaja Hitam yang tidak bersenjata, mencetuskan jawatannya.<EOS>')

In [18]:
string = 'Religion usually means trust in God, or a supernatural and supernatural power like God, as well as practices and institutions associated with that belief. Religion and belief are two very relevant things. But Religion has a broader meaning, which refers to a system of cohesive belief, and this belief is about the divine aspect'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded],model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

('Agama biasanya bermaksud kepercayaan kepada Tuhan, atau kekuatan ghaib dan ghaib seperti Tuhan, serta amalan dan institusi yang berkaitan dengan kepercayaan itu. Agama dan kepercayaan adalah dua perkara yang sangat relevan. Tetapi Agama mempunyai makna yang lebih luas, yang merujuk kepada sistem kepercayaan kohesif, dan kepercayaan ini adalah mengenai aspek ilahi<EOS>',
 'Agama biasanya bermaksud kepercayaan kepada Tuhan, atau kekuatan ghaib dan ghaib seperti Tuhan, serta amalan dan institusi yang berkaitan dengan kepercayaan itu. Agama dan kepercayaan adalah dua perkara yang sangat relevan. Tetapi Agama mempunyai makna yang lebih luas, yang merujuk kepada sistem kepercayaan kohesif, dan kepercayaan ini adalah mengenai aspek ilahi<EOS><pad>')

In [19]:
string = 'dia ni, like this lah, we have to do like this'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded],model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

('dia ni, macam ni lah, kita kena buat macam ni.<EOS>',
 'dia ni, macam ni lah, kita kena buat macam ni.<EOS>')

In [20]:
string = 'Jadi haram jadah😀😃🤭'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded],model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

('Jadi haram jadah😀😃🤭<EOS>', 'Jadi haram jadah😀😃🤭<EOS><pad>')

In [21]:
string = 'nak gi mana tuu'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded],model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

('nak gi mana nak ke?<EOS>', 'nak gi mana nak ke?<EOS>')

In [22]:
string = 'i already bought the stuffs, why suddenly cancel it'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded],model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0]), encoder.decode(b[0])

('Saya sudah membeli barang-barang itu, mengapa tiba-tiba membatalkannya<EOS>',
 'Saya sudah membeli barang-barang itu, mengapa tiba-tiba membatalkannya<EOS>')

In [23]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'transformer-en-ms-base/model.ckpt')

'transformer-en-ms-base/model.ckpt'

In [24]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'alphas' in n.name
        or 'temperature' in n.name
        or 'beam_size' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'modality' not in n.name
        and 'Assign' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'beam_size',
 'temperature',
 'transformer/body/target_space_embedding/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/target_space_embedding/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_scale/Read/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_bias/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp_1',
 'transformer/body/encoder/layer_0/self_attention/multihead_attention/q/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/multihead_attention/q/Tensordot/ReadVariableOp',
 'transformer/bod

In [25]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [26]:
freeze_graph('transformer-en-ms-base', strings)

INFO:tensorflow:Restoring parameters from transformer-en-ms-base/model.ckpt


INFO:tensorflow:Restoring parameters from transformer-en-ms-base/model.ckpt


INFO:tensorflow:Froze 201 variables.


INFO:tensorflow:Froze 201 variables.


INFO:tensorflow:Converted 201 variables to const ops.


INFO:tensorflow:Converted 201 variables to const ops.


11017 ops in the final graph.


In [27]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [28]:
g = load_graph('transformer-en-ms-base/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
greedy = g.get_tensor_by_name('import/greedy:0')
beam = g.get_tensor_by_name('import/beam:0')
test_sess = tf.InteractiveSession(graph = g)

In [29]:
from tensorflow.tools.graph_transforms import TransformGraph

In [30]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_constants(ignore_errors=true)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

pb = 'transformer-en-ms-base/frozen_model.pb'
input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

print(pb)

transformed_graph_def = TransformGraph(input_graph_def, 
                                       ['Placeholder'],
                                       ['greedy', 'beam'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

transformer-en-ms-base/frozen_model.pb


2022-07-25 20:42:29.025267: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying add_default_attributes
2022-07-25 20:42:29.115036: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying remove_nodes
2022-07-25 20:42:29.185232: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for greedy
2022-07-25 20:42:29.185292: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for beam
2022-07-25 20:42:29.275985: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for greedy
2022-07-25 20:42:29.276131: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for beam
2022-07-25 20:42:29.373130: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for greedy
2022-07-25 20:42:29.373184: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for beam
2022-07-25 20:42:29.452483: I tensorflow/tools/graph_transforms/remove_nodes.cc:

In [31]:
string = "eat chicken loh."
encoded = encoder.encode(string) + [1]
g = test_sess.run([greedy], feed_dict = {x:[encoded]})

In [32]:
encoder.decode(g[0][0])

'makan loh ayam.<EOS>'

In [33]:
from malaya_boilerplate.huggingface import upload_dict

In [37]:
!rm transformer-en-ms-base/*.tfevents.*

In [38]:
# !cp t2t-noisy-en-ms/base/*.tfevents.* transformer-en-ms-base
!tar -cvf transformer-base-noisy-en-ms.tar transformer-en-ms-base

transformer-en-ms-base/
transformer-en-ms-base/checkpoint
transformer-en-ms-base/frozen_model.pb.quantized
transformer-en-ms-base/model.ckpt.index
transformer-en-ms-base/model.ckpt.data-00000-of-00001
transformer-en-ms-base/model.ckpt.meta
transformer-en-ms-base/frozen_model.pb


In [39]:
files_mapping = {'transformer-base-noisy-en-ms.tar': 'transformer-base-noisy-en-ms.tar'}
upload_dict(model = 'pretrained-translation', files_mapping = files_mapping)

In [40]:
files_mapping = {'transformer-en-ms-base/frozen_model.pb': 'model.pb'}
upload_dict(model = 'translation-en-ms-noisy-base', files_mapping = files_mapping)

In [41]:
files_mapping = {'transformer-en-ms-base/frozen_model.pb.quantized': 'model.pb'}
upload_dict(model = 'translation-en-ms-noisy-base-quantized', files_mapping = files_mapping)